In [2]:
%run -i "preamble.py"

In [4]:
import numpy as np
import pandas as pd
import re
df = pd.read_csv("data/jobs_galvani_2023.csv", sep="|")

In [ ]:
# GO THROUGH THE DATASET COLUMN FOR COLUMN
# ANALYSE WHAT IS WRONG / FILTER OUT / CONVERT DTYPE
# SUMMARY WHAT HAPPENED IS AT THE END

In [5]:
## JobID Column
# Filter for "Real Allocations" (Types 1 and 6) (See Notion)
def get_job_id_type(jid):
    jid = str(jid)
    if jid.isdigit():
        return "Standard Integer (e.g. 633)"
    elif '.' in jid:
        # Check suffix to distinguish batch/extern from numeric steps
        suffix = jid.split('.')[-1]
        if suffix.isdigit():
            return "Job Step: Numeric (e.g. 633.0)"
        else:
            return f"Job Step: String (e.g. 633.{suffix})"
    elif '[' in jid and ']' in jid:
        return "Array Range Summary (e.g. 5419_[0-10])"
    elif '_' in jid:
        return "Array Task Instance (e.g. 5419_1)"
    else: return "Other / Unknown"

# 1. Apply classification
job_types = df['JobID'].apply(get_job_id_type)
# 2. Create a summary
summary = df.groupby(job_types)['JobID'].agg(
    Count='count',
    Examples=lambda x: x.sample(3, random_state=1).tolist() if len(x) > 3 else x.tolist()
).reset_index()
print(summary)

# Logic: Keep rows where JobID has NO dots AND NO brackets
mask_real_allocations = (
    ~df['JobID'].str.contains(r'\.', regex=True) & 
    ~df['JobID'].str.contains(r'\[', regex=True)
)
print(f"Total Rows in data before cutting: {len(df)}")
df = df[mask_real_allocations].copy()
print(f"Total Unique Allocations (Types 1 & 6): {len(df)}")
df.head()

                                    JobID   Count  \
0  Array Range Summary (e.g. 5419_[0-10])     204   
1       Array Task Instance (e.g. 5419_1)  367252   
2          Job Step: Numeric (e.g. 633.0)  187308   
3       Job Step: String (e.g. 633.batch)  822078   
4      Job Step: String (e.g. 633.extern)  840399   
5             Standard Integer (e.g. 633)  625527   

                                            Examples  
0  [461955_[0-2], 950721_[0-2], 129984_[2-1000%10...  
1                  [47373_66, 354747_17, 241296_132]  
2              [929269_126.0, 668671_396.0, 41287.7]  
3     [259051_201.batch, 642109.batch, 975711.batch]  
4  [322471.extern, 724826_864.extern, 852388_4.ex...  
5                           [318629, 477486, 842684]  
Total Rows in data before cutting: 2842768
Total Unique Allocations (Types 1 & 6): 992779


,JobID,Partition,State,ExitCode,Submit,Start,End,Elapsed,AllocTRES,ConsumedEnergyRaw
0,1,galvani-cpu,FAILED,0:105,2023-11-13T04:55:31,2023-11-13T04:55:31,2023-11-13T04:55:34,00:00:03,"billing=1,cpu=1,mem=1138967M,node=1",0.0
2,2,galvani-cpu,FAILED,0:105,2023-11-13T04:56:14,2023-11-13T04:56:14,2023-11-13T04:56:17,00:00:03,"billing=1,cpu=1,mem=1138967M,node=1",0.0
4,3,galvani-cpu,FAILED,0:105,2023-11-13T04:58:18,2023-11-13T04:58:18,2023-11-13T04:58:21,00:00:03,"billing=1,cpu=1,mem=1138967M,node=1",0.0
6,4,galvani-cpu,FAILED,0:105,2023-11-13T04:58:57,2023-11-13T04:58:57,2023-11-13T04:59:00,00:00:03,"billing=1,cpu=1,mem=1138967M,node=1",0.0
8,5,galvani-cpu,FAILED,0:105,2023-11-13T05:00:04,2023-11-13T05:00:04,2023-11-13T05:00:07,00:00:03,"billing=1,cpu=1,mem=1138967M,node=1",0.0


In [6]:
## Partition column
# Filter invalid partitions
print("--- Unique Partitions ---")
print(df['Partition'].unique())

print("\n--- Partition Counts ---")
print(df['Partition'].value_counts())

valid_partitions = [
    '2080-galvani', 
    '2080-preemptable-galvani', 
    'a100-galvani',
    'cpu-galvani',
    'a100-preemptable-galvani',
    'a100-fat-galvani',
]
df = df[df['Partition'].isin(valid_partitions)].copy()
print("\n--- Filtered Partition Counts ---")
print(df['Partition'].value_counts())
print(f"Total Rows after partition filtering: {len(df)}")

--- Unique Partitions ---
['galvani-cpu' 'galvani-2080' 'cpu-galvani' '2080-galvani' 'a100-galvani'
 '2080-preemptable-galvani' 'bethge'
 '2080-preemptable-galvani,cpu-galvani,bethge' 'a100-galvani,2080-galvani'
 'bethge,2080-galvani' '2080-preemptable-galvani,bethge'
 'bethge,2080-preemptable-galvani' 'bethge,a100-galvani,2080-galvani'
 'testing' 'testing,bethge' 'services' 'macke' 'a100-preemptable-galvani'
 'a100-fat-galvani' '2080-galvani,bethge' 'mladm']

--- Partition Counts ---
Partition
2080-galvani                                   305350
2080-preemptable-galvani                       255400
a100-galvani                                   210227
cpu-galvani                                    163636
a100-preemptable-galvani                        32162
a100-fat-galvani                                15353
bethge                                           7839
macke                                             776
2080-preemptable-galvani,cpu-galvani,bethge       734
2080-preemptab

In [7]:
## State column
# ARE THESE USER IDS? IS THIS SENSITIVE DATA?
print("\n--- Top 20 Most Common States ---")
print(df['State'].value_counts().head(10))
print("\n--- Sample of 'Weird' States ---")
# Filter for states that are unusually long (likely containing " by user")
weird_states = df[df['State'].astype(str).str.len() > 15]['State'].unique()
print(weird_states[:10])

# Unify all unique "cancels" into one "cancel"
df['State'] = df['State'].astype(str).str.split().str[0]
print("\n--- Top 20 Most Common States After Unifying ---")
print(df['State'].value_counts().head(20))
df.head()


--- Top 20 Most Common States ---
State
COMPLETED            540297
FAILED               222967
CANCELLED by 4393     28030
CANCELLED by 4119     19135
TIMEOUT               18054
CANCELLED by 4708     11473
CANCELLED by 4877      9573
CANCELLED by 4519      9175
CANCELLED by 4930      9067
CANCELLED by 4710      7334
Name: count, dtype: int64

--- Sample of 'Weird' States ---
['CANCELLED by 4635' 'CANCELLED by 1000' 'CANCELLED by 4321'
 'CANCELLED by 4205' 'CANCELLED by 4573' 'CANCELLED by 4423'
 'CANCELLED by 4532' 'CANCELLED by 4430' 'CANCELLED by 4537'
 'CANCELLED by 4517']

--- Top 20 Most Common States After Unifying ---
State
COMPLETED        540297
FAILED           222967
CANCELLED        198184
TIMEOUT           18054
OUT_OF_MEMORY      1998
PREEMPTED           557
NODE_FAIL            44
REQUEUED             27
Name: count, dtype: int64


,JobID,Partition,State,ExitCode,Submit,Start,End,Elapsed,AllocTRES,ConsumedEnergyRaw
32,18,cpu-galvani,COMPLETED,0:0,2023-11-13T07:39:03,2023-11-13T07:39:03,2023-11-13T07:39:04,00:00:01,"billing=1,cpu=1,mem=1138967M,node=1",0.0
34,19,2080-galvani,COMPLETED,0:0,2023-11-13T07:39:14,2023-11-13T07:39:14,2023-11-13T07:39:14,00:00:00,"billing=2,cpu=2,mem=354566M,node=1",0.0
36,20,2080-galvani,COMPLETED,0:0,2023-11-13T07:39:21,2023-11-13T07:39:21,2023-11-13T07:39:21,00:00:00,"billing=2,cpu=2,mem=354566M,node=1",0.0
38,21,2080-galvani,COMPLETED,0:0,2023-11-13T07:39:41,2023-11-13T07:39:41,2023-11-13T07:39:41,00:00:00,"billing=2,cpu=2,mem=354566M,node=1",0.0
40,22,cpu-galvani,COMPLETED,0:0,2023-11-13T07:39:45,2023-11-13T07:39:45,2023-11-13T07:39:45,00:00:00,"billing=1,cpu=1,mem=1138967M,node=1",0.0


In [8]:
## ExitCode
# Split 'ExitCode' (e.g., "1:0") into two distinct integer columns
# Create a new df_failures with cols: 
#           
# ReturnCode | Signal | State | Count

temp_df = df[['ExitCode', 'State']].copy()
split_codes = temp_df['ExitCode'].astype(str).str.split(':', expand=True)
temp_df['ReturnCode'] = pd.to_numeric(split_codes[0], errors='coerce').fillna(0).astype(int)
temp_df['Signal'] = pd.to_numeric(split_codes[1], errors='coerce').fillna(0).astype(int)
df_failures = (
    temp_df.groupby(['ReturnCode', 'Signal', 'State'])
    .size()
    .reset_index(name='Count')
)
df_failures = df_failures[df_failures['Count'] > 0].sort_values('Count', ascending=False).reset_index(drop=True)
print(df_failures)

# What to do with this info?
# What to the different return codes / signal tell us?

    ReturnCode  Signal          State   Count
0            0       0      COMPLETED  540297
1            0       0      CANCELLED  198160
2            1       0         FAILED  173790
3            0       0        TIMEOUT   18051
4            2       0         FAILED   12231
5            0      53         FAILED   11322
6          120       0         FAILED   10258
7            9       0         FAILED    6267
8          127       0         FAILED    3912
9            0       9         FAILED    3106
10           0     125  OUT_OF_MEMORY    1998
11           0       0      PREEMPTED     557
12          11       0         FAILED     493
13          15       0         FAILED     411
14           6       0         FAILED     286
15          64       0         FAILED     227
16           0       2         FAILED     202
17          10       0         FAILED     126
18           7       0         FAILED      63
19          39       0         FAILED      47
20          13       0         FAI

In [9]:
## Submit column
# 1. Look for entries that don't match the right format
# 2. Convert entries to a "datetime" object (https://docs.python.org/3/library/datetime.html)
# 3. Replace nasty entries with a "NaT" ("not a time") object instead (https://pandas.pydata.org/docs/reference/api/pandas.NaT.html)

# The regex for "YYYY-MM-DDTHH:MM:SS"
# \d{4} = 4 digits, \d{2} = 2 digits, T is the separator
iso_pattern = r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}$'
submit_strs = df['Submit'].dropna().astype(str)
# Find values that DO NOT match the pattern
mask_invalid = ~submit_strs.str.match(iso_pattern)
nasty_entries = submit_strs[mask_invalid]
print(f"Total rows: {len(df)}")
print(f"Valid ISO timestamps: {len(submit_strs) - len(nasty_entries)}")
print(f"Invalid/Nasty entries: {len(nasty_entries)}")
if len(nasty_entries) > 0:
    print("\n--- Top 'Nasty' Formats Found ---")
    print(nasty_entries.value_counts().head(10))
else:
    print("\nAll non-null values match the correct format!")

# Convert to datetime
df['Submit'] = pd.to_datetime(df['Submit'], errors='coerce')
print("\n--- Sample Values ---")
print(df['Submit'].head())

Total rows: 982128
Valid ISO timestamps: 982128
Invalid/Nasty entries: 0

All non-null values match the correct format!

--- Sample Values ---
32   2023-11-13 07:39:03
34   2023-11-13 07:39:14
36   2023-11-13 07:39:21
38   2023-11-13 07:39:41
40   2023-11-13 07:39:45
Name: Submit, dtype: datetime64[ns]


In [10]:
## Start column
# 1. Look for entries that don't match the right format
# 2. Convert entries to a "datetime" object (https://docs.python.org/3/library/datetime.html)
# 3. Replace nasty entries with a "NaT" ("not a time") object instead (https://pandas.pydata.org/docs/reference/api/pandas.NaT.html)

# The regex for "YYYY-MM-DDTHH:MM:SS"
# \d{4} = 4 digits, \d{2} = 2 digits, T is the separator
iso_pattern = r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}$'
submit_strs = df['Start'].dropna().astype(str)
# Find values that DO NOT match the pattern
mask_invalid = ~submit_strs.str.match(iso_pattern)
nasty_entries = submit_strs[mask_invalid]
print(f"Total rows: {len(df)}")
print(f"Valid ISO timestamps: {len(submit_strs) - len(nasty_entries)}")
print(f"Invalid/Nasty entries: {len(nasty_entries)}")
if len(nasty_entries) > 0:
    print("\n--- Top 'Nasty' Formats Found ---")
    print(nasty_entries.value_counts().head(10))
else:
    print("\nAll non-null values match the correct format!")

# Convert to datetime
df['Start'] = pd.to_datetime(df['Start'], errors='coerce')
print("\n--- Sample Values ---")
print(df['Start'].head())

# What are the "Unknown" entries? Jobs that got cancelled before they could start?

Total rows: 982128
Valid ISO timestamps: 836963
Invalid/Nasty entries: 0

All non-null values match the correct format!

--- Sample Values ---
32   2023-11-13 07:39:03
34   2023-11-13 07:39:14
36   2023-11-13 07:39:21
38   2023-11-13 07:39:41
40   2023-11-13 07:39:45
Name: Start, dtype: datetime64[ns]


In [11]:
## End column
# 1. Look for entries that don't match the right format
# 2. Convert entries to a "datetime" object (https://docs.python.org/3/library/datetime.html)
# 3. Replace nasty entries with a "NaT" ("not a time") object instead (https://pandas.pydata.org/docs/reference/api/pandas.NaT.html)

# The regex for "YYYY-MM-DDTHH:MM:SS"
# \d{4} = 4 digits, \d{2} = 2 digits, T is the separator
iso_pattern = r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}$'
submit_strs = df['End'].dropna().astype(str)
# Find values that DO NOT match the pattern
mask_invalid = ~submit_strs.str.match(iso_pattern)
nasty_entries = submit_strs[mask_invalid]
print(f"Total rows: {len(df)}")
print(f"Valid ISO timestamps: {len(submit_strs) - len(nasty_entries)}")
print(f"Invalid/Nasty entries: {len(nasty_entries)}")
if len(nasty_entries) > 0:
    print("\n--- Top 'Nasty' Formats Found ---")
    print(nasty_entries.value_counts().head(10))
else:
    print("\nAll non-null values match the correct format!")

# Convert to datetime
df['End'] = pd.to_datetime(df['End'], errors='coerce')
print("\n--- Sample Values ---")
print(df['End'].head())

# What are the "Unknown" entries? Why are there 128, but only 91 for start?

Total rows: 982128
Valid ISO timestamps: 982128
Invalid/Nasty entries: 0

All non-null values match the correct format!

--- Sample Values ---
32   2023-11-13 07:39:04
34   2023-11-13 07:39:14
36   2023-11-13 07:39:21
38   2023-11-13 07:39:41
40   2023-11-13 07:39:45
Name: End, dtype: datetime64[ns]


In [12]:
## Elapsed column
# Convert the string into pandas timedelta format (https://pandas.pydata.org/docs/reference/api/pandas.Timedelta.html)

# Format looks like this:
# Short jobs (< 24h): MM:SS (rarely) or HH:MM:SS.
# Long jobs (> 24h): Days-HH:MM:SS (e.g., 1-12:00:00 for 36 hours).
# Regex Explanation:
# ^          : Start of string
# (\d+-)?    : Optional "Days" part (digits followed by a dash), e.g., "1-"
# \d{2}:\d{2}:\d{2} : Standard HH:MM:SS
# $          : End of string
elapsed_pattern = r'^(\d+-)?\d{2}:\d{2}:\d{2}$'
elapsed_strs = df['Elapsed'].dropna().astype(str)
mask_invalid = ~elapsed_strs.str.match(elapsed_pattern)
nasty_entries = elapsed_strs[mask_invalid]
print(f"Total rows: {len(df)}")
print(f"Valid Elapsed formats: {len(elapsed_strs) - len(nasty_entries)}")
print(f"Invalid/Nasty entries: {len(nasty_entries)}")

if len(nasty_entries) > 0:
    print("\n--- Top 'Nasty' Formats Found ---")
    print(nasty_entries.value_counts().head(10))
else:
    print("\nAll non-null values match the correct format!")

# 1. Pre-process the string: Replace the hyphen '-' with ' days '
# We use regex=False for speed since we just want a literal replace
clean_elapsed = df['Elapsed'].astype(str).str.replace('-', ' days ', regex=False)
# 2. Convert to Timedelta
df['Elapsed'] = pd.to_timedelta(clean_elapsed, errors='coerce')
print("\n--- Sample Values (Converted) ---")
print(df['Elapsed'].head())
print("\n--- Longest Running Job ---")
print(df['Elapsed'].max())

Total rows: 982128
Valid Elapsed formats: 982128
Invalid/Nasty entries: 0

All non-null values match the correct format!

--- Sample Values (Converted) ---
32   0 days 00:00:01
34   0 days 00:00:00
36   0 days 00:00:00
38   0 days 00:00:00
40   0 days 00:00:00
Name: Elapsed, dtype: timedelta64[ns]

--- Longest Running Job ---
30 days 00:00:19


In [13]:
## AllocTres column
# What Parameters exist at all?

tres_series = df['AllocTRES'].dropna().astype(str)
# Step A: Split by comma (creates a list of "key=value" strings)
# Step B: Explode (turns the list into separate rows, expanding the dataframe)
# Step C: Split by '=' and take the first part (the key)
tres_keys = (tres_series.str.split(',').explode().str.split('=').str[0])
tres_counts = tres_keys.value_counts().reset_index()
tres_counts.columns = ['Parameter', 'Count']
tres_counts['Frequency (%)'] = (tres_counts['Count'] / len(df) * 100).round(2)
print("--- Unique AllocTRES Parameters ---")
print(tres_counts)

--- Unique AllocTRES Parameters ---
            Parameter   Count  Frequency (%)
0                 cpu  831955          84.71
1                 mem  831955          84.71
2                node  831955          84.71
3            gres/gpu  666406          67.85
4             billing  475254          48.39
5  gres/gpu:rtx2080ti  470603          47.92
6       gres/gpu:a100  195803          19.94


In [ ]:
## NEEDS CHANGE

## AllocTres column
# Are "gres/gpu" and "gres/gpu:h100" identical in their entries?

gpu_generic = df['AllocTRES'].str.extract(r'gres/gpu=(\d+)').astype(float)
gpu_specific = df['AllocTRES'].str.extract(r'gres/gpu:h100=(\d+)').astype(float)
# Check if they are exactly equal (handling NaNs)
# We use .equals() which considers NaN == NaN as True
are_identical = gpu_generic.equals(gpu_specific)
print(f"Are 'gres/gpu' and 'gres/gpu:h100' values identical? {are_identical}")
if not are_identical:     # If different, show us where
    diff_mask = (gpu_generic[0] != gpu_specific[0]) & (~gpu_generic[0].isna() | ~gpu_specific[0].isna())
    print(df.loc[diff_mask, 'AllocTRES'].head())

Are 'gres/gpu' and 'gres/gpu:h100' values identical? True


In [ ]:
## AllocTres column
# Split parameters into these distinct columns
# AllocCPUs | AllocGPUs (H100) | AllocNodes | AllocMem (MB) | Billing
# unify "gres/gpu" and"gres/gpu:h100" as they are identical

def extract_tres_param(df, col_name, pattern):
    # Regex explanation:
    # pattern     : The specific key we are looking for (e.g. 'cpu')
    # .*?         : Non-greedy match for extra chars (handles 'gres/gpu:h100')
    # =           : The separator
    # ([^,]+)     : Capture everything that is NOT a comma (the value)
    regex = rf"{pattern}.*?=([^,]+)"
    df[col_name] = df['AllocTRES'].astype(str).str.extract(regex, expand=False).fillna('UNKNOWN')
# 1. AllocCPUs (Look for 'cpu=')
extract_tres_param(df, 'AllocCPUs', 'cpu')
# 2. AllocGPUs (Look for 'gres/gpu')
# This regex 'gres/gpu.*?' will catch either 'gres/gpu=' OR 'gres/gpu:h100='
# Since they are identical, extracting the first match is sufficient.
extract_tres_param(df, 'AllocGPUs (H100)', 'gres/gpu')
# 3. AllocNodes (Look for 'node=')
extract_tres_param(df, 'AllocNodes', 'node')
# 4. AllocMem (Look for 'mem=')
extract_tres_param(df, 'AllocMem (MB)', 'mem')
# 5. Billing (Look for 'billing=')
extract_tres_param(df, 'Billing', 'billing')
df.head()

,JobID,Partition,State,ExitCode,Submit,Start,End,Elapsed,AllocTRES,ConsumedEnergyRaw,AllocCPUs,AllocGPUs (H100),AllocNodes,AllocMem (MB),Billing
0,633,h100-ferranti,FAILED,1:0,2024-10-15 13:36:43,2024-10-15 13:36:43,2024-10-15 13:36:43,0 days 00:00:00,"billing=208,cpu=1,gres/gpu:h100=8,gres/gpu=8,m...",0.0,1,8,1,2063800M,208
2,634,h100-ferranti,FAILED,1:0,2024-10-15 13:37:07,2024-10-15 13:37:07,2024-10-15 13:37:07,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,UNKNOWN,1,2063800M,208
4,635,h100-ferranti,FAILED,1:0,2024-10-15 13:37:12,2024-10-15 13:37:12,2024-10-15 13:37:12,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,UNKNOWN,1,2063800M,208
6,636,h100-ferranti,FAILED,1:0,2024-10-15 13:38:27,2024-10-15 13:38:27,2024-10-15 13:38:27,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,UNKNOWN,1,2063800M,208
8,637,h100-ferranti,FAILED,127:0,2024-10-15 13:42:37,2024-10-15 13:42:37,2024-10-15 13:42:48,0 days 00:00:11,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,UNKNOWN,1,2063800M,208


In [ ]:
## AllocCpus
# Convert to integer. Replace Unkowns with 0.

print("--- Top 5 CPU Allocations ---")
print(df['AllocCPUs'].value_counts().head())
# 1. Convert to numeric, turning 'Unkowns' into NaN
df['AllocCPUs'] = pd.to_numeric(df['AllocCPUs'], errors='coerce')
# 2. Fill NaN with 0 and convert to Integer
df['AllocCPUs'] = df['AllocCPUs'].fillna(0).astype(int)

# Why do we have so many unknowns? 29314

--- Top 5 CPU Allocations ---
AllocCPUs
2          207252
UNKNOWN     29314
8           25802
4           13181
16          11152
Name: count, dtype: int64


In [ ]:
## AllocGpus (H100)
# Convert to integer. Replace Unkowns with 0.

print("--- Top 5 GPUs (H100) Allocations ---")
print(df['AllocGPUs (H100)'].value_counts().head())
# 1. Convert to numeric, turning 'Unkowns' into NaN
df['AllocGPUs (H100)'] = pd.to_numeric(df['AllocGPUs (H100)'], errors='coerce')
# 2. Fill NaN with 0 and convert to Integer
df['AllocGPUs (H100)'] = df['AllocGPUs (H100)'].fillna(0).astype(int)

# Why do we have so many unknowns? That's almost 80%!

--- Top 5 GPUs (H100) Allocations ---
AllocGPUs (H100)
UNKNOWN    234255
1           45832
2            7139
4            7102
8            1440
Name: count, dtype: int64


In [ ]:
## AllocNodes
# Convert to integer. Replace Unkowns with 0.

print("--- Top 5 Nodes Allocations ---")
print(df['AllocNodes'].value_counts().head())
# 1. Convert to numeric, turning 'Unkowns' into NaN
df['AllocNodes'] = pd.to_numeric(df['AllocNodes'], errors='coerce')
# 2. Fill NaN with 0 and convert to Integer
df['AllocNodes'] = df['AllocNodes'].fillna(0).astype(int)

# Why do we have so many unknowns? 29315

--- Top 5 Nodes Allocations ---
AllocNodes
1          267273
UNKNOWN     29315
2             167
4              23
8              12
Name: count, dtype: int64


In [ ]:
## AllocMem (MB)
# Convert to Integer. Convert to MB. Replace Unknowns with 0.

# 1. Extract the Numeric value (including decimals) and the Unit letter
# Regex Breakdown:
# (?P<val>[\d\.]+) : Capture digits and dots into a group named 'val'
# (?P<unit>[MGT])  : Capture M, G, or T into a group named 'unit'
mem_parts = df['AllocMem (MB)'].astype(str).str.extract(r'(?P<val>[\d\.]+)(?P<unit>[MGT])')

# 2. Define the multipliers to convert everything to MB
# G = 1024 MB, T = 1024*1024 MB
multipliers = {
    'M': 1,
    'G': 1024,
    'T': 1024 * 1024
}
# 3. Map the multipliers to the extracted unit
mem_parts['mult'] = mem_parts['unit'].map(multipliers)
# 4. Calculate MB: (Float Value) * (Multiplier)
# pd.to_numeric handles the string-to-float conversion
# rows that were "UNKNOWN" or didn't match the regex become NaN automatically here
df['AllocMem (MB)'] = pd.to_numeric(mem_parts['val']) * mem_parts['mult']
# 5. Clean up: Fill NaNs (UNKNOWNs) with 0 and convert to Integer
df['AllocMem (MB)'] = df['AllocMem (MB)'].fillna(0).astype(int)

# If logic worked, T values should be > 1,000,000
print("\n--- Check logic on 'Big' jobs (Terabytes) ---")
print(df[df['AllocMem (MB)'] > 1000000]['AllocMem (MB)'].head())


--- Check logic on 'Big' jobs (Terabytes) ---
0    2063800
2    2063800
4    2063800
6    2063800
8    2063800
Name: AllocMem (MB), dtype: int64


In [ ]:
## Billing
# Convert to integers. Replace Unknowns with 0.

# 1. Isolate the column and filter out explicitly missing data
# We use a mask so we can look at the "Rest"
mask_unknown = df['Billing'] == 'UNKNOWN'
billing_data = df.loc[~mask_unknown, 'Billing'].astype(str)
print(f"Total Rows: {len(df)}")
print(f"UNKNOWN Billing: {mask_unknown.sum()}")
# 2. Check for Non-Integers
# Regex: ^\d+$ matches only strings composed entirely of digits (0-9)
# We look for things that do NOT match (~)
non_integers = billing_data[~billing_data.str.match(r'^\d+$')]
if len(non_integers) > 0:
    print(f"\nfound {len(non_integers)} non-integer values!")
    print("Examples:", non_integers.head().tolist())
else:    print("\n All non-UNKNOWN values are valid integers.")
# 3. Convert to Integer (Robustly)
# Coerce errors -> NaN, Fill NaN -> 0, Convert to Int
df['Billing'] = pd.to_numeric(df['Billing'], errors='coerce').fillna(0).astype(int)
# 4. Count Zeros
# Note: This includes the original '0's AND the 'UNKNOWN's we just filled
zeros_count = (df['Billing'] == 0).sum()
print(f"\nFinal Zeros Count (converted): {zeros_count}")
print(f"  -> {mask_unknown.sum()} were 'UNKNOWN' (filled as 0)")
original_zeros = (billing_data.astype(int) == 0).sum() if len(billing_data) > 0 else 0
print(f"  -> {original_zeros} were explicitly '0'")

Total Rows: 296798
UNKNOWN Billing: 78107

 All non-UNKNOWN values are valid integers.

Final Zeros Count (converted): 78107
  -> 78107 were 'UNKNOWN' (filled as 0)
  -> 0 were explicitly '0'


In [ ]:
# To Recap what we did. We went through each Column:

# JobID:            Only kept Array Task and Main Alloc
# Partition:        Removed three weird partitions (should we remove h1001-ferranti?)
# State:            Removed the User ids (?) -> unify into one "CANCELLED" state
# ExitCode:         Created df_failurse - split ExitCodes into ReturnCode and Signal. Not sure what to make with that?
# Submit:           Converted to datetime format. No Unknowns.
# Start:            Converted to datetime format. 91 Unknowns.
# End:              Converted to datetime format. 128 Unknowns.
# Elapsed:          Converted to timedelta format. No Unknowns.
# AllocTres:        Split up into new columns: [AllocCPUs | AllocGPUs (H100) | AllocNodes | AllocMem (MB) | Billing]
# AllocCPUs:        Convert to integer. Replace Unkowns with 0.
# AllocGpus (H100): Convert to integer. Replace Unkowns with 0.
# AllocNodes:       Convert to integer. Replace Unkowns with 0.
# AllocMem (MB):    Convert to integer. Convert to MB. Replace Unknowns with 0.
# Billing:          Convert to integers. Replace Unknowns with 0.

In [ ]:
# Final df #
summary_rows = []
for col in df.columns:
    n_unique = df[col].nunique(dropna=False)
    dtype = df[col].dtype
    summary_rows.append((col, n_unique, dtype))
summary_df = pd.DataFrame(summary_rows, columns=["Column", "Unique values", "Dtype"])
print("Unique values and Dtype")
print(summary_df.to_string(index=False))
print()
df.head(10)

Unique values and Dtype
           Column  Unique values           Dtype
            JobID         296798          object
        Partition              3          object
            State              9          object
         ExitCode             31          object
           Submit          47505  datetime64[ns]
            Start         173710  datetime64[ns]
              End         241904  datetime64[ns]
          Elapsed          19229 timedelta64[ns]
        AllocTRES            859          object
ConsumedEnergyRaw              2         float64
        AllocCPUs             39           int64
 AllocGPUs (H100)             12           int64
       AllocNodes              8           int64
    AllocMem (MB)            129           int64
          Billing            138           int64



,JobID,Partition,State,ExitCode,Submit,Start,End,Elapsed,AllocTRES,ConsumedEnergyRaw,AllocCPUs,AllocGPUs (H100),AllocNodes,AllocMem (MB),Billing
0,633,h100-ferranti,FAILED,1:0,2024-10-15 13:36:43,2024-10-15 13:36:43,2024-10-15 13:36:43,0 days 00:00:00,"billing=208,cpu=1,gres/gpu:h100=8,gres/gpu=8,m...",0.0,1,8,1,2063800,208
2,634,h100-ferranti,FAILED,1:0,2024-10-15 13:37:07,2024-10-15 13:37:07,2024-10-15 13:37:07,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
4,635,h100-ferranti,FAILED,1:0,2024-10-15 13:37:12,2024-10-15 13:37:12,2024-10-15 13:37:12,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
6,636,h100-ferranti,FAILED,1:0,2024-10-15 13:38:27,2024-10-15 13:38:27,2024-10-15 13:38:27,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
8,637,h100-ferranti,FAILED,127:0,2024-10-15 13:42:37,2024-10-15 13:42:37,2024-10-15 13:42:48,0 days 00:00:11,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
10,638,h100-ferranti,FAILED,1:0,2024-10-15 13:43:08,2024-10-15 13:43:08,2024-10-15 13:43:08,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
12,639,h100-ferranti,FAILED,1:0,2024-10-15 13:44:43,2024-10-15 13:44:43,2024-10-15 13:44:43,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
14,640,h100-ferranti,FAILED,1:0,2024-10-16 12:34:58,2024-10-16 12:34:58,2024-10-16 12:34:59,0 days 00:00:01,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
16,641,h100-ferranti,FAILED,1:0,2024-10-16 12:35:29,2024-10-16 12:35:29,2024-10-16 12:35:29,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
18,642,h100-ferranti,FAILED,1:0,2024-10-16 12:37:32,2024-10-16 12:37:32,2024-10-16 12:37:32,0 days 00:00:00,"billing=208,cpu=1,mem=2063800M,node=1",0.0,1,0,1,2063800,208
